<a href="https://colab.research.google.com/github/davinun99/tesis-model/blob/main/RadomBalancing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First load the dataset

In [14]:
import pandas as pd
pd.set_option('display.max_columns', None) #MOSTRAR TODAS LAS COLUMNAS
!gdown --id 1-tNBUCWFU0_b9QW6ie4Vvfs7eb4WbIPY
# https://drive.google.com/file/d/1-tNBUCWFU0_b9QW6ie4Vvfs7eb4WbIPY/view?usp=share_link
df_completo = pd.read_csv('TestBalanceo1.csv', sep=";;;", index_col=False)

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1-tNBUCWFU0_b9QW6ie4Vvfs7eb4WbIPY
To: /content/TestBalanceo1.csv
100% 369M/369M [00:01<00:00, 197MB/s]


<ipython-input-14-9b6e16741e0d>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_completo = pd.read_csv('TestBalanceo1.csv', sep=";;;", index_col=False)


In [20]:
df_complaints = df_completo[(df_completo['has_complaint'] == True)]
df_no_complaints = df_completo[(df_completo['has_complaint'] == False)]
n_cmp = len(df_complaints.index)
n_no_cmp = len(df_no_complaints.index)

# print('total: ', len(df_completo.index))
# print('complaints: ', n_cmp)
# print('no complaints: ', n_no_cmp)

df_no_complaints = df_no_complaints.sample(int(n_cmp*3/2)) #Sample 60% no complaints 40 % complaints
n_no_cmp = len(df_no_complaints.index)

print('total: ', len(df_completo.index))
print('complaints: ', n_cmp)
print('no complaints: ', n_no_cmp)

total:  97262
complaints:  10332
no complaints:  15498


In [21]:
result = pd.concat([df_complaints, df_no_complaints])
print('result length: ', len(result.index))

result length:  25830


Now prepare H2O

In [24]:
! apt-get install default-jre
!java -version
!pip install h2o
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
from sklearn.model_selection import train_test_split

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-jre is already the newest version (2:1.11-72).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
openjdk version "11.0.19" 2023-04-18
OpenJDK Runtime Environment (build 11.0.19+7-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.19+7-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
df = result
x_train, x_test = train_test_split(df, test_size=0.15, random_state=17)
import h2o
from h2o.automl import H2OAutoML
h2o.__version__

'3.40.0.4'

In [29]:
h2o.init(ip="127.0.0.1", max_mem_size_GB = 200, nthreads = 10)
hf = h2o.H2OFrame(x_train)
# hf.head()
y_label = "has_complaint"
x_labels = list(df.drop(columns=[y_label]).columns)
# hf[y_label] = hf[y_label].asfactor()
aml = H2OAutoML(max_runtime_secs=60*5)
# %%time
aml.train(x = x_labels, y = y_label, training_frame = hf)

Checking whether there is an H2O instance running at http://127.0.0.1:54321. connected.


H2O_cluster_uptime:,3 mins 41 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.4
H2O_cluster_version_age:,22 days
H2O_cluster_name:,H2O_from_python_unknownUser_43z2sn
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,199.8 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
20:15:58.187: _train param, Dropping bad and constant columns: [contracts.amendments.amendsAmount_usd, parties.details.EntityType candidate, parties.details.legalEntityTypeDetail buyer, parties.details.EntityType enquirer, parties.details.legalEntityTypeDetail procuringEntity, contracts.amendments.amendsAmount_pyg, language, parties.details.EntityType supplier, parties.details.EntityType notifiedSupplier, initiationType, parties.details.EntityType tenderer, parties.details.EntityType payee, parties.details.legalEntityTypeDetail payer, id, tag, ocid, ocid.1, tender.coveredBy]

█████████████████████
20:17:34.148: _train param, Dropping bad and constant columns: [contracts.amendments.amendsAmount_usd, parties.details.EntityType candidate, parties.details.legalEntityTypeDetail buyer, parties.details.EntityType enquirer, parties.details.legalEntityTypeDetail procuringEntity, con

H2OConnectionError: ignored